# Session 4 - Hands on Examples

<div class="alert alert-success">
 
**By the end of this session you will be able to**

- Apply the code to invert a DMA response function loaded from a data file.
- Apply the code to invert size resolved CCN data loaded from a data file.

</div>


<div class="alert alert-danger">

**Instructions**
    
- Go to Menu "Cell" -> "Run All Below"
- The notebook will initialize all cells 
- Navigate back to this point and work through the notebook sequentially.
- When finished, move to Session 5.
 
</div>

## 1. Inverting Size Distribution Data

In [ ]:
include("Scripts/startup.jl")
play("Audio/examples1.ogg")


<img src="Figures/nbs7_f01.png" width="300" align = "left">
<img src="Figures/example_data.png" width="300" align = "middle">
<br><br>



In [ ]:
# Load a simple comma delimited text file
df = CSV.read("example_data.csv", DataFrame)

In [ ]:
play("Audio/examples2.ogg")

The DMA was operated at 20C and 940 hPa, with 4 L min-1 sheath flow, and 1 L min-1 sample flow. The DMA geometry is the TSI cylindrical long column, operated with a positive polarity power supply. The inversion is carried out to 6 charges. Transmission efficiency through the DMA is unity. The DMA is gridded to 120 bins and the grid is spanning the full range of the DMA's capability (10V to 10kV). Use setupSMPS instead of setupDMA if you want to use the SMPS transfer function.

In [ ]:
t, p, lpm = 293.15, 940e2, 1.666e-5      # Temperature [K], Pressure [Pa], L min-1 to m3 s-1
r₁, r₂, l = 9.37e-3,1.961e-2,0.44369     # DMA geometry [m]
Λ = DMAconfig(t,p,1lpm,4lpm,r₁,r₂,l,0.0,:+,6,:cylindrical)  

In [ ]:
play("Audio/examples3.ogg")

In [ ]:
@time δ = setupDMA(Λ, vtoz(Λ,10000), vtoz(Λ,10), 120);

<div class="alert alert-warning">

The @time macro benchmarks the grid creation. Most of the time is spent for precomputing the various inversion matrices. This is a one-time cost.
    
</div>


<div class="alert alert-danger">

**DMPS vs. SMPS**
    
To use the SMPS transfer function instead of the DMPS transfer function you can setup the grid using the setupSMPS function. The function takes the arguments: DMAConfig, Vmax, Vmin, scan time, bin accumulation time.
    
```julia
@time δ = setupSMPS(Λ, 10000, 10, 120, 1.0)
```
    
Evaluating the matrix for the SMPS is 10 times slower than for the DMPS setup. This is because the transfer function itself is an integral. The time to perform the inversion per scan is similar for DMPS and SMPS.
    
</div>

In [ ]:
play("Audio/examples4.ogg")

In [ ]:
    𝕣 = (df,:Dp,:Rcn,δ) |> interpolateDataFrameOntoδ;

<div class="alert alert-warning">

This function takes the columns :Dp and :Rcn from the DataFrame df and creates a response distribution of the type SizeDistribution that matches the DMA grid thisδ. 
    
</div>

In [ ]:
play("Audio/examples5.ogg")

In [ ]:
𝐈, 𝐒, 𝐀, λ =  δ.𝐈, δ.𝐒, δ.𝐀, 0.5
@time 𝕟ⁱⁿᵛ¹ = (𝐀'𝐀 + λ^2𝐈)^(-1) * (𝐀'𝕣  + λ^2 * 𝐒^(-1)*𝕣);

<div class="alert alert-warning">

Compute regularized inverse with prescribed λ. This is fast. 
    
</div>

In [ ]:
play("Audio/examples6.ogg")

In [ ]:
@time 𝕟ⁱⁿᵛ² = rinv(𝕣.N, δ, λ₁=0.1, λ₂=1.0);

<div class="alert alert-warning">

- Computing the regularized inverse using the L-curve method is slow. This is a recurring cost per inverted size distribution. 
- The inversion is forced to run on a single thread (default in julia currently is 8). For this matrix equation, increasing the number of threads will lead to performance degradation. 
- Installing the MKL library (https://github.com/JuliaComputing/MKL.jl) provides a small performance gain over OpenBLAS default that is shipped with Julia.
    
    
</div>

In [ ]:
play("Audio/examples7.ogg")

<div class="alert alert-danger">

The function rinv performs the inversion using the internal matrices. It is straightforward to use a custom matrix in the inversion. The L-curve inversion is performed in these steps

- setup the system
- run the lcorner function which finds the corner along the L-curve
- compute the size distribution
- create a SizeDistribution object
    
**Let's perform an inversion on our distribution using a custom matrix.**
    
</div>

In [ ]:
# Transmission function for DMA without loss function - swap out functions for your own case
T(zˢ,k,Λ,δ) = δ.Ω(Λ,δ.Z,zˢ/k).*δ.Tc(k,δ.Dp)

# Transmission function for thisΛ, thisδ DMA and only using single charged particles
my𝐀 = (hcat(map(zˢ->Σ(k->T(zˢ,k,Λ,δ),1),δ.Z)...))'

setupRegularization(my𝐀,δ.𝐈,𝕣.N,inv(δ.𝐒)*𝕣.N,1)  # setup the system of equations to regularize using my𝐀
λopt = lcorner(0.01,1.0;n=10,r=3)                    # compute the optimal λ using recursive algorithn
N = clean((reginv(λopt, r = :Nλ))[1])                # find the inverted size distribution 
𝕟ⁱⁿᵛ³ = SizeDistribution([],δ.De,δ.Dp,δ.ΔlnD,N./δ.ΔlnD,N,:regularized);

In [ ]:
play("Audio/examples8.ogg")

In [ ]:
include("Scripts/plot_data_inversion.jl")

**Figure.** Left: raw response function from data file. Right: results from the three inversions. Note that 𝕟ⁱⁿᵛ³ only considers singly charged particles, thus showing an overestimate.

<div class="alert alert-success">

**You have met the following learning objective.**

- Apply the code to invert a DMA response function loaded from a data file.
    
</div>

## 2. Inverting Size-resolved CCN Measurements

In [ ]:
play("Audio/examples9.ogg")

Size resolved CCN measurements have a long history (e.g. Cruz and Pandis, 1997, Snider et al., 2006). Particles are typically dried, charge neutralized, and passed through the DMA. At the exit, the flow is split between a CPC that measures all particles and a CCN counter that measures particles that form cloud droplets at a water supersaturation set by the instrument. In this configuration the DMA voltage can either be stepped or continuously scanned. The ratio of CCN-to-CPC response function is used to determine the fraction of particles that activate at a given diameter. The diameter where 50% of the particles activate is taken to be the activation diameter. The activation of particles into cloud droplets is proportional to the volume of solute in the particle. Therefore, larger multiply charged particles activate first. This leads to a bias in the inferred D50 diameter if the activated fraction is computed from the ratio of the raw response functions (Petters et al., 2007, 2009)

### 2.1 Loading and Conditioning Data

In [ ]:
play("Audio/examples10.ogg")

In [ ]:
# Load a simple comma delimited text file - file contains :Dp, :Rcn, :Rccn
# Note that this is the same DMA/size distribution defined by this DMA
df = CSV.read("example_data.csv")
𝕣ᶜⁿ = (df,:Dp,:Rcn,δ) |> interpolateDataFrameOntoδ        # CN response distribution
𝕣ᶜᶜⁿ = (df,:Dp,:Rccn,δ) |> interpolateDataFrameOntoδ;     # CCN response distribution

In [ ]:
play("Audio/examples11.ogg")

<div class="alert alert-danger">
    
Thresholding is performed for bins with low counts, which results in NaN and Inf in 𝕒𝕗. This is necessary because at low concentrations a bin may have zero or very low concentration, resulting in unrealistic or NaN/InF values in the activated fraction array. Thus if counts/concentration is below the threshold, the activated fraction is forced to one for large diameters and zero for small diameters. The noise threshold may need to be adjusted for different datasets.
    
</div>

In [ ]:
function threshold!(𝕟::SizeDistribution, c::Float64, n1::Float64, n2::Float64)
   N = 𝕟.N  
   S = 𝕟.S
   S[(N .<= c) .& (𝕟.Dp .> 150)] .= n2
   N[(N .<= c) .& (𝕟.Dp .> 150)] .= n2
   S[(N .<= c) .& (𝕟.Dp .< 150)] .= n1
   N[(N .<= c) .& (𝕟.Dp .< 150)] .= n1
   𝕟.N = N
end

threshold!(𝕣ᶜⁿ, 0.1, 0.1, 0.1)
threshold!(𝕣ᶜᶜⁿ, 0.1, 0.0, 0.1)

𝕒𝕗 = 𝕣ᶜᶜⁿ/𝕣ᶜⁿ;

In [ ]:
play("Audio/examples12.ogg")

In [ ]:
include("Scripts/plot_ccn_data.jl")

**Figure**. Left: Raw CN and CCN response functions. Right: activated fraction defined as the ratio of the response functions. The figure  shows  a shoulder of particles that appear to activating early. These are +2 charged particles transmitting through the DMA that influence of +2 the observed ratio 𝕣ᶜᶜⁿ./𝕣ᶜⁿ.

### 2.2 Modeling the Actived Fraction

In [ ]:
play("Audio/examples13.ogg")

The CCN transmission function is modeled using a cumulative Gauss integral <br>
<center> $T_{af} = \frac{1}{2}\left[1 + \mathrm{erf}\left(\frac{d-\mu}{\sigma}\right) \right] $ </center>
This function is applied to the mobility size distribution. Then the response function is computed. Empirically, activated fraction can be computed using from the ratio of size distributions and response functions. 

In [ ]:
# CCN activation unction 
#  - 𝕟 is a size distribution from which the diameter vector is used
#  - μ is the activation diameter
#  - σ is the spread
Taf(𝕟,μ,σ) = @. 0.5 * (1.0 + erf((𝕟.Dp - μ)./(sqrt(2.0σ))));

In [ ]:
play("Audio/examples14.ogg")

To predict the ratio of 𝕣ᶜᶜⁿ/𝕣ᶜⁿ in terms of the activated fraction model Taf:<br><br>

<center> $𝕟^{\mathrm{cn}} = (\bf{A}^\rm{T}\bf{A} + \lambda^\rm{2} \bf{I})^\rm{-1}(\bf{A}^\rm{T} \rm{𝕣^{\mathrm{cn}}} + \lambda^2\bf{S}^{-1} \rm{𝕣^{\mathrm{cn}}})$ </center>
<br>
<center> $\left( \frac{𝕣^{\mathrm{ccn}}}{𝕣^{\mathrm{cn}}} \right)_{\mathrm{model}} = \frac{\mathbf{A}[T_{af}\; .*\; 𝕟^{\mathrm{cn}}]}{\mathbf{A}𝕟^{\mathrm{cn}}}$ </center>

<br>

This mirrors the approach in Petters et al. (2007). The model + fit can be solved in few lines of code.

```julia
𝕟ᶜⁿ = (𝐀'𝐀 + λ^2𝐈)^(-1) * (𝐀'𝕣ᶜⁿ  + λ^2 * 𝐒^(-1)*𝕣ᶜⁿ)
model(x,p) = (δ.𝐀*(𝕟ᶜⁿ.N.*Taf(𝕟ᶜⁿ, p[1], p[2])))./(δ.𝐀*𝕟ᶜⁿ.N)
fit = curve_fit(model, δ.Dp, 𝕣ᶜᶜⁿ.N./𝕣ᶜⁿ.N, [65.0, 3.0])
```

In [ ]:
𝐈, 𝐒, 𝐀, λ =  δ.𝐈, δ.𝐒, δ.𝐀, 0.5
𝕟ᶜⁿ = (𝐀'𝐀 + λ^2𝐈)^(-1) * (𝐀'𝕣ᶜⁿ  + λ^2 * 𝐒^(-1)*𝕣ᶜⁿ)
model(x,p) = (𝐀 * (𝕟ᶜⁿ.N .* Taf(𝕟ᶜⁿ, p[1], p[2])))./( 𝐀 * 𝕟ᶜⁿ.N)
fit = curve_fit(model, 𝕒𝕗.Dp, 𝕒𝕗.N, [65.0, 3.0])
Ax = fit.param
afmodel = model(δ.Dp, Ax)
DataFrame(μ=round(Ax[1],digits=1), σ=round(Ax[2],digits=1))

<div class="alert alert-warning">
    
The predicted activation diameter is 86.4 nm. 
    
</div>

In [ ]:
play("Audio/examples15.ogg")

In [ ]:
include("Scripts/plot_ccn_data_fit.jl")

**Figure**. Left: Raw CN and CCN response functions. Right: activated fraction. The afmodel accurately describes the +2 charge shoulder.

<div class="alert alert-danger">
    
- The +2 charge shoulder can in some cases be large and interfere with the measurement
- It is also possible to fit Taf to 𝕟ᶜⁿ/𝕟ᶜᶜⁿ, where both distributions are inverted. This results in extra noise since two inversions are performed.
- The model is only  weakly sensitive to the choice of λ. For small λ, negative values might need filtering. Use of the L-curve is not needed in most cases.
- This example is also in a clean notebook: "Inversion of Size Resolved CCN Data From Data Example"
    
</div>

<div class="alert alert-success">

**You have met the following learning objective.**

- Apply the code to invert size resolved CCN data loaded from a data file.
    
</div>